In [3]:
import torch
import einops
import math

from dataclasses import dataclass

from utils.config import NetworkConfig, WorldConfig


torch.distributed.optim.zero_redundancy_optimizer.ZeroRedundancyOptimizer

In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
def exists(val):
    return val is not None

def default(val, d):
    return val if exists(val) else d


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')